# Benchmarks

## Prep 

In [1]:
from cnn.utils.graph_manipulation import load_frozen_graph, predict_from_frozen
from cnn.utils.dataset import load_cifar10, dataset_preprocessing_by_keras

/home/daibak/.virtualenvs/aca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pyperf = !find ~ -iname pyperf 2>/dev/null
pyperf = pyperf[0]

In [3]:
nets = ["dense", "conv", "medium"]
names = [[n + "_opt", n + "_quant"] for n in nets]
frozen_nets = [["models/" + n + ".pb"  for n in both] for both in names]
results_files = [["results/" + n + ".json"  for n in both] for both in names]

## Check accuracy 

In [4]:
_, _, x_test, t_test = load_cifar10()
x_test = dataset_preprocessing_by_keras(x_test)

In [ ]:
for pb_files in frozen_nets:
    for m in pb_files:
        graph = load_frozen_graph(m)
        out = predict_from_frozen(graph, [x_test], ["features"], ["classes:0", "softmax:0"])
        classes = np.concatenate([batch[0] for batch in out])
        acc = accuracy_score(np.argmax(t_test, axis=1), classes)
    print(f"{m}: {acc} accuracy")

## Time comparisons 

In [ ]:
for r in results_files:
    !{pyperf} show {r[0]} --quiet

In [ ]:
for r in results_files:
    !{pyperf} compare_to {r[0]} {r[1]} --table